In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline  

In [4]:
data = pd.read_csv('./gifts-data.tsv', delimiter='\t')
data = data.drop([i for i in range (1, len(data), 2)], axis=0)

In [103]:
t = 340
data['OUTPUT:self_gift_name'].str.lower().str.replace('-', '').str.strip().unique()[t:t+20]

array(['корзинка для цветов', 'парфюм, косметика',
       'декоративная подушка с надписью.', 'свитер легкий',
       'блонот под заказ', 'косметика по уходу за лицом',
       'духи шанель номер 5', 'статуэтка', 'кухонные весы', 'зонтик',
       'альбом с фотографиями', 'игровые наушники',
       'принадлежности для ванны', 'зимняя шуба', 'собака',
       'лошадиный дилдо (нет серьезно, не шучу)', 'комнатные тапки',
       'книга "дачный дом..."', 'самодельное пирожное',
       'метакса 12 звезд'], dtype=object)

In [56]:
# drop 'джіпіес' 'эестетический' 'подарок' 'comfort' 'подарок от чистого сердца,сделанный своими руками'
mapping = {'смартфон': 'телефон',
           'колье':    'украшение',
           'часы casio': 'часы',
           'несколько шоколадок, его любимое печенье': 'шоколад, печенье',
           'францая духи названия не помню':'духи',
           'духи hugo boss':'духи',
           'духи hugo boss deep red':'духи',
            'часы наручные':'часы',
             'подарочная карта в магазин рив гош':'подарочная карта',
             'браслет':'украшение',
             'ручуа':'ручка',
             'картина пейзаж':'картина',
             'мобильный телефон bq stanford и парфюм': 'телефон',
             'это был планшет, сделанный на заказ.': 'планшет',
             'айфон': 'телефон',
              'парфюм ': 'духи',
              'ювелирное украшение': 'украшение',
              'парфюм': 'духи',
              'цепочка с кулоном': 'украшение',
              'сертификат в магазин косметики': 'подарочная карта',
              'это были чехол на паспорт, майка с рисунком волка, рюкзак': 'чехол на паспорт',
          'macbook pro': 'ноутбук',
           'маленькая мягкая игрушка': 'мягкая игрушка',
            'картина с его любимым автомобилем, по его фотографии, нарисованная мной': 'картина',
            'часы мужские дизель': 'часы',
            'sony xperia z': 'телефон',
            'оригинальная футболка fila': 'футболка',
            'смартчаси': 'часы',
            '2 теплых свитера ванс': 'свитер',
            'смарт часы': 'часы',
            'bluetooth наушник': 'наушники',
             'драгоценное украшение': 'украшение',
             'самсунг галакси': 'телефон',
             'игрушечная бетономешалка на пульте ду) . так как у моего парня бизнес с бетономешалками': 'машинка на пульте управления'
            }

In [102]:
def map_(x):
    if x in mapping.keys():
        return mapping[x]
    elif 'парфюм' in x or 'духи' in x or 'туалетная' in x:
        return 'духи'
    elif 'планшет' in x:
        return 'планшет'
    elif 'ноутбук' in x or 'нетбук' in x or 'тлефнр' in x:
        return 'ноутбук'
    elif 'украшение' in x or 'браслет' in x or 'кулон' in x or 'цепоч' in x \
        or 'подвеск' in x or 'золот' in x or 'серебр' in x or 'колечко' in x:
        return 'украшение'
    elif 'часы' in x or 'watch' in x:
        return 'часы'
    elif 'запонк' in x:
        return 'запонки'
    elif 'телефон' in x or 'смартфон' in x or 'phone' in x \
         or 'самсунг' in x or 'samsung' in x or 'xiaomi' in x:
        return 'телефон'
    elif 'мягк' in x or 'медв' in x:
        return 'мягкая игрушка'
    elif 'белье' in x:
        return 'белье'
    elif 'удочка' in x or 'спиннинг' in x or 'удилище' in x or 'спининг' in x:
        return 'удочка'
    elif 'игра' in x:
        return 'игра'
    elif 'наушник' in x:
        return 'наушники'
    elif 'крем' in x or 'космет' in x:
        return 'уходовая косметика'
    elif 'delongi' in x:
        return 'кофемашина'
    elif 'банк' in x:
        return 'павербанк'
    elif 'портрет' in x:
        return 'картина'
    elif 'колонк' in x:
        return 'колонка'
    elif 'котенок' in x or 'катенок' in x or 'кошка' in x or 'котёнок' in x:
        return 'кошка'
    else:
        return x
len(data['OUTPUT:self_gift_name'].str.lower().str.replace('-', '').str.strip().apply(map_).unique())

413